In [1]:
from src.model import CNN,  ModelMixBert, BertModelClassification
from torch import nn
from src.data_load.data_loader import load_split_data
from torch.optim import SGD, Adam
from torch.utils.tensorboard import SummaryWriter
from src.utils.utils import get_device, plot_images
from transformers import BertModel        
from torchvision import models



import json
import torch

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

import warnings
warnings.filterwarnings("ignore")

In [3]:
from src.train_model import Train

BATCH_SIZE = 32
LEARN_RATE = 0.001
EMBEDDING_DIM = 200

with open('./datainfo.json', 'r') as f:
    data = json.load(f)  
    
vocab = data["vocab"]
vocab_size = len(vocab)
writer = SummaryWriter(comment=f"LSTM-128 {LEARN_RATE} {BATCH_SIZE}")

trainloader, testloader, data_model = load_split_data(data, BATCH_SIZE, .2, bert = True)
vocab_size

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


KeyboardInterrupt: 

In [ ]:

device = get_device()

model_ft = models.resnet50(pretrained=True)
for parameter in model_ft.parameters():
    parameter.requires_grad = True

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 256)
model_ft = model_ft.to(device)

model_bert = BertModel.from_pretrained('bert-base-uncased')
for parameter in model_bert.parameters():
    parameter.requires_grad = True

model1 = BertModelClassification(model_bert, 256)

model2 = CNN(256)
model = ModelMixBert(model2, model1, 512, 3)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=LEARN_RATE, momentum=0.9)

In [ ]:
experiment = Train(model, optimizer, criterion,trainloader, testloader, writer = writer, show_matrix=True, show_image=False, epochs=10, include_text=True, show_metrics=True)
experiment.train_bert(include_image=True)